In [1]:
#import necessary packages
from pathlib import Path
import os
import ast
import shutil
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, fbeta_score, f1_score
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import LocalOutlierFactor
from sktime.transformations.panel.rocket import (
    MiniRocket,
    MiniRocketMultivariate,
    MiniRocketMultivariateVariable,
)

In [2]:
#collection of dataset information for each telemetry data 
training_dfs = []
test_dfs = []

training_df = []
test_df = []

file_names = []

label_df = pd.read_csv("labeled_anomalies.csv")

train_data_path = "npy_train" 
test_data_path = "npy_test" 

os.makedirs("raw_train", exist_ok=True)
os.makedirs("raw_test", exist_ok=True)

index = 0
for root, _, files in os.walk(train_data_path):
    for file in files:
        example_path = os.path.join(root, file)
        filename = Path(example_path).stem
        
        row_indices = label_df[label_df['chan_id'] == filename].index.tolist()
        if not row_indices:
            continue
        spacecraft = label_df.loc[row_indices[0], 'spacecraft']
        
        if (spacecraft == "SMAP"): 
            data = np.load(example_path)
            df = pd.DataFrame(data)
            print("(train) df: ", filename, df.shape, "index: ", index)
            df.to_csv(f"raw_train/{filename}.csv", index=False)
            training_dfs.append(df) 
            index += 1

index = 0
for root, _, files in os.walk(test_data_path):
    for file in files:
        example_path = os.path.join(root, file)
        filename = Path(example_path).stem
        
        row_indices = label_df[label_df['chan_id'] == filename].index.tolist()
        if not row_indices:
            continue
        spacecraft = label_df.loc[row_indices[0], 'spacecraft']
        
        if (spacecraft == "SMAP"): 
            data = np.load(example_path)
            df = pd.DataFrame(data)
            print("(test) df: ", filename, df.shape, "index: ", index)
            test_dfs.append(df) 
            file_names.append(filename)
            
            index += 1

In [2]:
def get_data():
    training_dfs = {}
    test_dfs = {}
    file_names = {"train": {}, "test": {}}
    label_df = pd.read_csv("labeled_anomalies.csv")
    train_data_path = "raw_train" 
    test_data_path = "raw_test" 
    
    for root, _, files in os.walk(train_data_path):
        for file in files:
            example_path = os.path.join(root, file)
            filename = Path(example_path).stem
            df = pd.read_csv(example_path)
            channel = filename[0]            
            if (training_dfs.get(channel) == None):
                training_dfs[channel] = []
                file_names["train"][channel] = []
                training_dfs[channel].append(df)
                file_names["train"][channel].append(filename)
            else:
                training_dfs[channel].append(df)
                file_names["train"][channel].append(filename)
    
    for root, _, files in os.walk(test_data_path):
        for file in files:
            example_path = os.path.join(root, file)
            filename = Path(example_path).stem
            df = pd.read_csv(example_path)
            channel = filename[0]
            if (test_dfs.get(channel) == None):
                test_dfs[channel] = []
                file_names["test"][channel] = []
                test_dfs[channel].append(df)
                file_names["test"][channel].append(filename)
            else:
                test_dfs[channel].append(df)
                file_names["test"][channel].append(filename)

    return (training_dfs, test_dfs, label_df, file_names)

In [3]:
def create_windows(training_dfs, test_dfs, window_size, window_overlap, label_df, file_names):
    training_data = {}
    test_data = {}
    label_data = {}
    
    for channel in training_dfs.keys():
        training_data[channel] = []
        for df in training_dfs[channel]: 
            for i in range(0, len(df), window_overlap):
                window = []
                if i + window_size > len(df): 
                    window = df.iloc[-window_size:].to_numpy().tolist()
                else: 
                    window = df.iloc[i:i + window_size].to_numpy().tolist()
                np_window = np.array(window)
                transposed_window = np_window.T
                normal_window = transposed_window.tolist() 
                training_data[channel].append(normal_window)
                
    for channel in test_dfs.keys():
        test_data[channel] = []
        label_data[channel] = []
        for i in range(len(test_dfs[channel])):
            df = test_dfs[channel][i]
            for j in range(0, len(df), window_overlap):
                window = []
                if j + window_size > len(df): 
                    window = df.iloc[-window_size:].to_numpy().tolist()
                else: 
                    window = df.iloc[j:j + window_size].to_numpy().tolist()
                row_indices = label_df[label_df["chan_id"] == file_names["test"][channel][i]].index.tolist()
                
                if not row_indices:
                    continue

                anomaly_sequence = label_df.loc[row_indices[0], 'anomaly_sequences']
                anomaly_sequence = ast.literal_eval(anomaly_sequence)
                labeled = False
                for anomalies in anomaly_sequence:
                    if (not(anomalies[1] <= j or anomalies[0] >= j + window_size)) and labeled == False:
                        label_data[channel].append(1)
                        labeled = True
                if labeled == False:
                    label_data[channel].append(0)
                np_window = np.array(window)
                transposed_window = np_window.T
                normal_window = transposed_window.tolist() 
                test_data[channel].append(normal_window)
                
    return (training_data, test_data, label_data)

In [9]:
window_size = 60 
window_overlap = 20
training_dfs, test_dfs, label_df, file_names = get_data()

X_train_collection, X_test_collection, y_test_collection = create_windows(training_dfs, test_dfs, window_size, window_overlap, label_df, file_names)

data = {}
os.makedirs("data", exist_ok=True)

for channel in X_train_collection:
    if data.get(channel) is None:
        data[channel] = {}
    if data.get(channel).get("train") is None:
        X_train = np.array(X_train_collection[channel])
        path = f"data/{channel}/train"
        os.makedirs(path, exist_ok=True)
        np.save(path + f"/{channel}.npy", X_train)
        data[channel]["train"] = path + f"/{channel}.npy"
        print("train: ", channel, "shape: ", X_train.shape)
    else:
        print("Error: duplicate training data sets")
   
    
for channel in X_test_collection: 
    if data.get(channel) is None:
        continue
    if data.get(channel).get("test") is None:
        X_test = np.array(X_test_collection[channel])
        path = f"data/{channel}/test"
        os.makedirs(path, exist_ok=True)
        np.save(path + f"/{channel}.npy", X_test)
        data[channel]["test"] = path + f"/{channel}.npy"
        print("test: ", channel, "shape: ", X_test.shape)
    else:
        print("Error: duplicate testing data sets")
   
  
for channel in y_test_collection:
    if data.get(channel) is None:
        continue
    if data.get(channel).get("label") is None:
        y_test = np.array(y_test_collection[channel])
        path = f"data/{channel}/label"
        os.makedirs(path, exist_ok=True)
        np.save(path + f"/{channel}.npy", y_test)
        data[channel]["label"] = path + f"/{channel}.npy"
        print("label: ", channel, "shape: ", y_test.shape)
    else:
        print("Error: duplicate label data sets")
    

train:  A shape:  (842, 25, 60)
train:  P shape:  (703, 25, 60)
train:  F shape:  (432, 25, 60)
train:  D shape:  (1445, 25, 60)
train:  R shape:  (144, 25, 60)
train:  G shape:  (793, 25, 60)
train:  T shape:  (431, 25, 60)
train:  E shape:  (1867, 25, 60)
train:  S shape:  (141, 25, 60)
train:  B shape:  (122, 25, 60)
test:  E shape:  (5486, 25, 60)
test:  T shape:  (1292, 25, 60)
test:  G shape:  (2405, 25, 60)
test:  D shape:  (4789, 25, 60)
test:  A shape:  (3374, 25, 60)
test:  F shape:  (1281, 25, 60)
test:  P shape:  (2056, 25, 60)
test:  B shape:  (403, 25, 60)
test:  R shape:  (363, 25, 60)
test:  S shape:  (367, 25, 60)
label:  E shape:  (5486,)
label:  T shape:  (1292,)
label:  G shape:  (2405,)
label:  D shape:  (4789,)
label:  A shape:  (3374,)
label:  F shape:  (1281,)
label:  P shape:  (2056,)
label:  B shape:  (403,)
label:  R shape:  (363,)
label:  S shape:  (367,)


In [2]:
channel_names = []

for channel in os.listdir("data"):
    if os.path.isdir(os.path.join("data", channel)):
        channel_names.append(channel)
print(channel_names)

['T', 'G', 'S', 'P', 'D', 'F', 'R', 'A', 'E', 'B']


In [5]:
raw_data= {}
for channel in channel_names:
    raw_data[channel] = {}
    training_data_path = f"data/{channel}/train/{channel}.npy"
    testing_data_path = f"data/{channel}/test/{channel}.npy"
    label_data_path = f"data/{channel}/label/{channel}.npy"
    
    X_train = np.load(training_data_path)
    X_test = np.load(testing_data_path)
    y_test = np.load(label_data_path)

    raw_data[channel]["train"] = X_train
    print("train: ", channel, "shape: ", X_train.shape)
    raw_data[channel]["test"] = X_test
    print("test: ", channel, "shape: ", X_test.shape)
    raw_data[channel]["label"] = y_test
    print("label: ", channel, "shape: ", y_test.shape)

train:  T shape:  (431, 25, 60)
test:  T shape:  (1292, 25, 60)
label:  T shape:  (1292,)
train:  G shape:  (793, 25, 60)
test:  G shape:  (2405, 25, 60)
label:  G shape:  (2405,)
train:  S shape:  (141, 25, 60)
test:  S shape:  (367, 25, 60)
label:  S shape:  (367,)
train:  P shape:  (703, 25, 60)
test:  P shape:  (2056, 25, 60)
label:  P shape:  (2056,)
train:  D shape:  (1445, 25, 60)
test:  D shape:  (4789, 25, 60)
label:  D shape:  (4789,)
train:  F shape:  (432, 25, 60)
test:  F shape:  (1281, 25, 60)
label:  F shape:  (1281,)
train:  R shape:  (144, 25, 60)
test:  R shape:  (363, 25, 60)
label:  R shape:  (363,)
train:  A shape:  (842, 25, 60)
test:  A shape:  (3374, 25, 60)
label:  A shape:  (3374,)
train:  E shape:  (1867, 25, 60)
test:  E shape:  (5486, 25, 60)
label:  E shape:  (5486,)
train:  B shape:  (122, 25, 60)
test:  B shape:  (403, 25, 60)
label:  B shape:  (403,)


In [3]:
transformed_data= {}
os.makedirs("transformed_data", exist_ok=True)

for channel in channel_names:
    transformed_data[channel] = {}
    training_data_path = f"data/{channel}/train/{channel}.npy"
    testing_data_path = f"data/{channel}/test/{channel}.npy"
    label_data_path = f"data/{channel}/label/{channel}.npy"
    
    X_train = np.load(training_data_path)
    X_test = np.load(testing_data_path)
    y_test = np.load(label_data_path)

    minirocket = MiniRocketMultivariate(num_kernels=10000, n_jobs = 2, random_state = 42) 
    minirocket.fit(X_train)
    X_transform_train = minirocket.transform(X_train)
    X_transform_test = minirocket.transform(X_test)

    transformed_data[channel]["train"] = X_transform_train
    print("train: ", channel, "shape: ", X_transform_train.shape)
    transformed_data[channel]["test"] = X_transform_test
    print("test: ", channel, "shape: ", X_transform_test.shape)
    transformed_data[channel]["label"] = y_test
    print("label: ", channel, "shape: ", y_test.shape)

for channel in transformed_data: 
    print("store: ", channel)
    X_transform_train = transformed_data[channel]["train"]
    X_transform_test = transformed_data[channel]["test"]
    y_test = transformed_data[channel]["label"]

    train_df = pd.DataFrame(X_transform_train)
    test_df = pd.DataFrame(X_transform_test)
    label_df = pd.DataFrame(y_test)

    training_data_path = f"transformed_data/{channel}/train"
    testing_data_path = f"transformed_data/{channel}/test"
    label_data_path = f"transformed_data/{channel}/label"
    
    os.makedirs(training_data_path, exist_ok=True)
    os.makedirs(testing_data_path, exist_ok=True)
    os.makedirs(label_data_path, exist_ok=True)

    train_df.to_csv(training_data_path + f"/{channel}.csv", index=False)
    test_df.to_csv(testing_data_path + f"/{channel}.csv", index=False)
    label_df.to_csv(label_data_path + f"/{channel}.csv", index=False)  

/home/DAVIDSON/haotaki/.local/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:373: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


train:  T shape:  (431, 9996)
test:  T shape:  (1292, 9996)
label:  T shape:  (1292,)
train:  G shape:  (793, 9996)
test:  G shape:  (2405, 9996)
label:  G shape:  (2405,)
train:  S shape:  (141, 9996)
test:  S shape:  (367, 9996)
label:  S shape:  (367,)
train:  P shape:  (703, 9996)
test:  P shape:  (2056, 9996)
label:  P shape:  (2056,)
train:  D shape:  (1445, 9996)
test:  D shape:  (4789, 9996)
label:  D shape:  (4789,)
train:  F shape:  (432, 9996)
test:  F shape:  (1281, 9996)
label:  F shape:  (1281,)
train:  R shape:  (144, 9996)
test:  R shape:  (363, 9996)
label:  R shape:  (363,)
train:  A shape:  (842, 9996)
test:  A shape:  (3374, 9996)
label:  A shape:  (3374,)
train:  E shape:  (1867, 9996)
test:  E shape:  (5486, 9996)
label:  E shape:  (5486,)
train:  B shape:  (122, 9996)
test:  B shape:  (403, 9996)
label:  B shape:  (403,)
stoe:  T
stoe:  G
stoe:  S
stoe:  P
stoe:  D
stoe:  F
stoe:  R
stoe:  A
stoe:  E
stoe:  B


In [3]:
min_max_fitted_data = {}

for channel in channel_names: 
    min_max_fitted_data[channel] = {}

    training_data_path = f"transformed_data/{channel}/train/{channel}.csv"
    testing_data_path = f"transformed_data/{channel}/test/{channel}.csv"
    label_data_path = f"transformed_data/{channel}/label/{channel}.csv"
    
    X_transform_train = pd.read_csv(training_data_path).to_numpy()
    X_transform_test =  pd.read_csv(testing_data_path).to_numpy()
    y_test = pd.read_csv(label_data_path).to_numpy()

    scaler = MinMaxScaler()
    scaler.fit(X_transform_train)
    X_fit_train = scaler.transform(X_transform_train)
    X_fit_test = scaler.transform(X_transform_test)
    
    min_max_fitted_data[channel]["train"] = X_fit_train
    min_max_fitted_data[channel]["test"] = X_fit_test
    min_max_fitted_data[channel]["label"] = y_test
    
    print("train: ", channel, "shape: ", X_fit_train.shape)
    print("test: ", channel, "shape: ", X_fit_test.shape)
    print("label: ", channel, "shape: ", y_test.shape)

train:  T shape:  (431, 9996)
test:  T shape:  (1292, 9996)
label:  T shape:  (1292, 1)
train:  G shape:  (793, 9996)
test:  G shape:  (2405, 9996)
label:  G shape:  (2405, 1)
train:  S shape:  (141, 9996)
test:  S shape:  (367, 9996)
label:  S shape:  (367, 1)
train:  P shape:  (703, 9996)
test:  P shape:  (2056, 9996)
label:  P shape:  (2056, 1)
train:  D shape:  (1445, 9996)
test:  D shape:  (4789, 9996)
label:  D shape:  (4789, 1)
train:  F shape:  (432, 9996)
test:  F shape:  (1281, 9996)
label:  F shape:  (1281, 1)
train:  R shape:  (144, 9996)
test:  R shape:  (363, 9996)
label:  R shape:  (363, 1)
train:  A shape:  (842, 9996)
test:  A shape:  (3374, 9996)
label:  A shape:  (3374, 1)
train:  E shape:  (1867, 9996)
test:  E shape:  (5486, 9996)
label:  E shape:  (5486, 1)
train:  B shape:  (122, 9996)
test:  B shape:  (403, 9996)
label:  B shape:  (403, 1)


In [3]:
standard_fitted_data = {}

for channel in channel_names: 
    standard_fitted_data[channel] = {}

    training_data_path = f"transformed_data/{channel}/train/{channel}.csv"
    testing_data_path = f"transformed_data/{channel}/test/{channel}.csv"
    label_data_path = f"transformed_data/{channel}/label/{channel}.csv"
    
    X_transform_train = pd.read_csv(training_data_path).to_numpy()
    X_transform_test =  pd.read_csv(testing_data_path).to_numpy()
    y_test = pd.read_csv(label_data_path).to_numpy()

    scaler = StandardScaler()
    scaler.fit(X_transform_train)
    X_fit_train = scaler.transform(X_transform_train)
    X_fit_test = scaler.transform(X_transform_test)
    
    standard_fitted_data[channel]["train"] = X_fit_train
    standard_fitted_data[channel]["test"] = X_fit_test
    standard_fitted_data[channel]["label"] = y_test
    
    print("train: ", channel, "shape: ", X_fit_train.shape)
    print("test: ", channel, "shape: ", X_fit_test.shape)
    print("label: ", channel, "shape: ", y_test.shape)

train:  T shape:  (431, 9996)
test:  T shape:  (1292, 9996)
label:  T shape:  (1292, 1)
train:  G shape:  (793, 9996)
test:  G shape:  (2405, 9996)
label:  G shape:  (2405, 1)
train:  S shape:  (141, 9996)
test:  S shape:  (367, 9996)
label:  S shape:  (367, 1)
train:  P shape:  (703, 9996)
test:  P shape:  (2056, 9996)
label:  P shape:  (2056, 1)
train:  D shape:  (1445, 9996)
test:  D shape:  (4789, 9996)
label:  D shape:  (4789, 1)
train:  F shape:  (432, 9996)
test:  F shape:  (1281, 9996)
label:  F shape:  (1281, 1)
train:  R shape:  (144, 9996)
test:  R shape:  (363, 9996)
label:  R shape:  (363, 1)
train:  A shape:  (842, 9996)
test:  A shape:  (3374, 9996)
label:  A shape:  (3374, 1)
train:  E shape:  (1867, 9996)
test:  E shape:  (5486, 9996)
label:  E shape:  (5486, 1)
train:  B shape:  (122, 9996)
test:  B shape:  (403, 9996)
label:  B shape:  (403, 1)


In [7]:
# performance bad for channels B, R, and S due to their small training data size
# nu - controls boundary tightness (Smaller nu → tighter boundary → fewer false positives, Larger nu → looser boundary → higher recall)
# threshold	- converts scores (anomaly / normal)

# Large (1000+ windows/channel)	Mostly clean	0.01 – 0.02
# Medium (hundreds of windows)	Mostly clean	0.02 – 0.05
# Small or noisy	Some anomalies in training	0.05 – 0.1
#tighter nu makes more sense as the training data is supposed to have only normal data - needs confirmation at this point in time 

anomaly_rate = []
for channel in standard_fitted_data: 
    print("channel: ", channel)
    anomaly_count = 0
    for label in standard_fitted_data[channel]["label"]:
        if label == 1:
            anomaly_count += 1
    anomaly_rate.append(anomaly_count / len(standard_fitted_data[channel]["label"]))
    print("anomaly_rate: ", anomaly_count / len(standard_fitted_data[channel]["label"]))

print(anomaly_rate)


channel:  T
anomaly_rate:  0.14628482972136223
channel:  G
anomaly_rate:  0.021205821205821207
channel:  S
anomaly_rate:  0.0681198910081744
channel:  P
anomaly_rate:  0.14348249027237353
channel:  D
anomaly_rate:  0.22635205679682607
channel:  F
anomaly_rate:  0.12724434035909446
channel:  R
anomaly_rate:  0.01928374655647383
channel:  A
anomaly_rate:  0.16478956727919383
channel:  E
anomaly_rate:  0.11009843237331389
channel:  B
anomaly_rate:  0.01488833746898263
[0.14628482972136223, 0.021205821205821207, 0.0681198910081744, 0.14348249027237353, 0.22635205679682607, 0.12724434035909446, 0.01928374655647383, 0.16478956727919383, 0.11009843237331389, 0.01488833746898263]


In [10]:
def predict(threshold, decision_scores):
    y_pred = []
    for score in decision_scores: 
        if score < threshold: 
            y_pred.append(1)
        else:
            y_pred.append(0)
    return np.array(y_pred)

In [37]:
def one_class_svm(channel_data, channel):
    print("channel:", channel)
    clf = OneClassSVM().fit(channel_data["train"])
    decision_scores = clf.decision_function(channel_data["test"])
    
    percentiles = np.arange(1, 30 + 1)
    thresholds = []
    for percent in percentiles: 
        thresholds.append(np.percentile(decision_scores, percent))
    
    # best_f1_score = 0
    best_scores = (0, 0, 0)  

    for th in thresholds:
        y_pred = predict(th, decision_scores)
        # f1 = f1_score(channel_data["label"], y_pred)
        recall = recall_score(channel_data["label"], y_pred, pos_label=1)
        precision = precision_score(channel_data["label"], y_pred, zero_division=0)
        if (recall > best_scores[0]) or (recall == best_scores[0] and precision > best_scores[1]):
            best_scores = (recall, precision, th)

    best_threshold = best_scores[2]
    
    y_test = channel_data["label"]
    y_pred = predict(best_threshold, decision_scores)
    print("best_threshold: ", best_threshold)
    print(classification_report(y_test, y_pred))
    precision = precision_score(y_test, y_pred, average='weighted')
    print("FDR:", 1 - precision)
    c_m = confusion_matrix(y_test, y_pred)
    print(c_m)

In [38]:
for channel in standard_fitted_data:
    one_class_svm(standard_fitted_data[channel], channel)

channel: T
best_threshold:  -0.8992676803852276
              precision    recall  f1-score   support

           0       0.83      0.68      0.75      1103
           1       0.10      0.21      0.14       189

    accuracy                           0.61      1292
   macro avg       0.47      0.44      0.44      1292
weighted avg       0.73      0.61      0.66      1292

FDR: 0.2732372204520219
[[754 349]
 [150  39]]
channel: G
best_threshold:  -3.894629500699023
              precision    recall  f1-score   support

           0       0.99      0.71      0.82      2354
           1       0.04      0.63      0.08        51

    accuracy                           0.71      2405
   macro avg       0.52      0.67      0.45      2405
weighted avg       0.97      0.71      0.81      2405

FDR: 0.03131591500501396
[[1664  690]
 [  19   32]]
channel: S
best_threshold:  -1.1175199796552189
              precision    recall  f1-score   support

           0       0.98      0.80      0.88      

In [39]:
def isolation_forest(channel_data, channel):
    print("channel:", channel)
    clf = IsolationForest(random_state=42, n_jobs=-1).fit(channel_data["train"])
    decision_scores = clf.decision_function(channel_data["test"])
    
    percentiles = np.arange(1, 30 + 1)
    thresholds = []
    for percent in percentiles: 
        thresholds.append(np.percentile(decision_scores, percent))
    
    # best_f1_score = 0
    best_scores = (0, 0, 0)  

    for th in thresholds:
        y_pred = predict(th, decision_scores)
        # f1 = f1_score(channel_data["label"], y_pred)
        recall = recall_score(channel_data["label"], y_pred, pos_label=1)
        precision = precision_score(channel_data["label"], y_pred, zero_division=0)
        if (recall > best_scores[0]) or (recall == best_scores[0] and precision > best_scores[1]):
            best_scores = (recall, precision, th)

    best_threshold = best_scores[2]
    
    y_test = channel_data["label"]
    y_pred = predict(best_threshold, decision_scores)
    print("best_threshold: ", best_threshold)
    print(classification_report(y_test, y_pred))
    precision = precision_score(y_test, y_pred, average='weighted')
    print("FDR:", 1 - precision)
    c_m = confusion_matrix(y_test, y_pred)
    print(c_m)

In [40]:
for channel in standard_fitted_data:
    isolation_forest(standard_fitted_data[channel], channel)

channel: T
best_threshold:  0.11642565349970714
              precision    recall  f1-score   support

           0       0.87      0.72      0.79      1103
           1       0.19      0.40      0.26       189

    accuracy                           0.67      1292
   macro avg       0.53      0.56      0.52      1292
weighted avg       0.77      0.67      0.71      1292

FDR: 0.22566689441851684
[[790 313]
 [114  75]]
channel: G
best_threshold:  0.03169341221337902
              precision    recall  f1-score   support

           0       0.98      0.71      0.82      2354
           1       0.02      0.22      0.03        51

    accuracy                           0.70      2405
   macro avg       0.50      0.46      0.43      2405
weighted avg       0.96      0.70      0.80      2405

FDR: 0.043807640922672086
[[1667  687]
 [  40   11]]
channel: S
best_threshold:  0.05234593268931187
              precision    recall  f1-score   support

           0       0.96      0.72      0.82   

In [21]:
#LOF learns a reference density from X_train, new points are compared against training density
def local_outlier_factor(channel_data, channel):
    print("channel:", channel)
    clf = LocalOutlierFactor(n_neighbors=50, novelty=True)
    clf.fit(channel_data["train"])
    decision_scores = clf.decision_function(channel_data["test"])
 
    percentiles = np.arange(1, 30 + 1)
    thresholds = []
    for percent in percentiles: 
        thresholds.append(np.percentile(decision_scores, percent))
    
    # best_f1_score = 0
    best_scores = (0, 0, 0)  

    for th in thresholds:
        y_pred = predict(th, decision_scores)
        # f1 = f1_score(channel_data["label"], y_pred)
        recall = recall_score(channel_data["label"], y_pred, pos_label=1)
        precision = precision_score(channel_data["label"], y_pred, zero_division=0)
        if (recall > best_scores[0]) or (recall == best_scores[0] and precision > best_scores[1]):
            best_scores = (recall, precision, th)

    best_threshold = best_scores[2]
    
    y_test = channel_data["label"]
    y_pred = predict(best_threshold, decision_scores)
    print("best_threshold: ", best_threshold)
    print(classification_report(y_test, y_pred))
    precision = precision_score(y_test, y_pred, average='weighted')
    print("FDR:", 1 - precision)
    c_m = confusion_matrix(y_test, y_pred)
    print(c_m)

In [22]:
for channel in standard_fitted_data:
    local_outlier_factor(standard_fitted_data[channel], channel)

channel: T
best_threshold:  0.48417609932572986
              precision    recall  f1-score   support

           0       0.85      0.71      0.78      1103
           1       0.15      0.29      0.20       189

    accuracy                           0.65      1292
   macro avg       0.50      0.50      0.49      1292
weighted avg       0.75      0.65      0.69      1292

FDR: 0.24958199270065595
[[783 320]
 [134  55]]
channel: G
best_threshold:  -0.07318787799478454
              precision    recall  f1-score   support

           0       0.99      0.76      0.86      2354
           1       0.05      0.61      0.10        51

    accuracy                           0.75      2405
   macro avg       0.52      0.68      0.48      2405
weighted avg       0.97      0.75      0.84      2405

FDR: 0.03096338683368427
[[1784  570]
 [  20   31]]
channel: S
best_threshold:  0.37770101237544224
              precision    recall  f1-score   support

           0       0.98      0.84      0.90   

In [33]:
#LOF learns a reference density from X_train, new points are compared against training density
def ensemble_local_outlier_factor(channel_data, channel):
    clf = LocalOutlierFactor(n_neighbors=50, novelty=True)
    clf.fit(channel_data["train"])
    decision_scores = clf.decision_function(channel_data["test"])
 
    percentiles = np.arange(1, 30 + 1)
    thresholds = []
    for percent in percentiles: 
        thresholds.append(np.percentile(decision_scores, percent))
    
    # best_f1_score = 0
    best_scores = (0, 0, 0)  

    for th in thresholds:
        y_pred = predict(th, decision_scores)
        # f1 = f1_score(channel_data["label"], y_pred)
        recall = recall_score(channel_data["label"], y_pred, pos_label=1)
        precision = precision_score(channel_data["label"], y_pred, zero_division=0)
        if (recall > best_scores[0]) or (recall == best_scores[0] and precision > best_scores[1]):
            best_scores = (recall, precision, th)

    best_threshold = best_scores[2]
    
    return predict(best_threshold, decision_scores)

In [34]:
def ensemble_isolation_forest(channel_data, channel):
    clf = IsolationForest(random_state=42, n_jobs=-1).fit(channel_data["train"])
    decision_scores = clf.decision_function(channel_data["test"])
    
    percentiles = np.arange(1, 30 + 1)
    thresholds = []
    for percent in percentiles: 
        thresholds.append(np.percentile(decision_scores, percent))
    
    # best_f1_score = 0
    best_scores = (0, 0, 0)  

    for th in thresholds:
        y_pred = predict(th, decision_scores)
        # f1 = f1_score(channel_data["label"], y_pred)
        recall = recall_score(channel_data["label"], y_pred, pos_label=1)
        precision = precision_score(channel_data["label"], y_pred, zero_division=0)
        if (recall > best_scores[0]) or (recall == best_scores[0] and precision > best_scores[1]):
            best_scores = (recall, precision, th)

    best_threshold = best_scores[2]
    
    return predict(best_threshold, decision_scores)
    

In [35]:
def ensemble_one_class_svm(channel_data, channel):
    clf = OneClassSVM().fit(channel_data["train"])
    decision_scores = clf.decision_function(channel_data["test"])
   
    percentiles = np.arange(1, 30 + 1)
    thresholds = []
    for percent in percentiles: 
        thresholds.append(np.percentile(decision_scores, percent))
    
    # best_f1_score = 0
    best_scores = (0, 0, 0)  

    for th in thresholds:
        y_pred = predict(th, decision_scores)
        # f1 = f1_score(channel_data["label"], y_pred)
        recall = recall_score(channel_data["label"], y_pred, pos_label=1)
        precision = precision_score(channel_data["label"], y_pred, zero_division=0)
        if (recall > best_scores[0]) or (recall == best_scores[0] and precision > best_scores[1]):
            best_scores = (recall, precision, th)

    best_threshold = best_scores[2]
    
    return predict(best_threshold, decision_scores)

In [36]:
for channel in standard_fitted_data: 
    y_pred_one_class_svm = ensemble_one_class_svm(standard_fitted_data[channel], channel)
    y_pred_isolation_forest = ensemble_isolation_forest(standard_fitted_data[channel], channel)
    y_pred_local_outlier_factor = ensemble_local_outlier_factor(standard_fitted_data[channel], channel)

    y_pred = []
    for i in range(len(y_pred_one_class_svm)): 
        ensemble = y_pred_one_class_svm[i] + y_pred_isolation_forest[i] + y_pred_local_outlier_factor[i]
        if ensemble >= 1: 
            y_pred.append(1)
        else:
            y_pred.append(0)
    print("channel: ",channel)
    y_test = (standard_fitted_data[channel]["label"])
    print(classification_report(y_test, y_pred))
    precision_1 = precision_score(y_test, y_pred, pos_label=1, zero_division=0)
    print("FDR:", 1 - precision)
    c_m = confusion_matrix(y_test, y_pred)
    print(c_m)

channel:  T
              precision    recall  f1-score   support

           0       0.86      0.35      0.50      1103
           1       0.15      0.67      0.25       189

    accuracy                           0.40      1292
   macro avg       0.51      0.51      0.37      1292
weighted avg       0.76      0.40      0.46      1292

FDR: 0.021250129834831766
[[385 718]
 [ 62 127]]
channel:  G
              precision    recall  f1-score   support

           0       0.99      0.52      0.69      2354
           1       0.04      0.80      0.07        51

    accuracy                           0.53      2405
   macro avg       0.51      0.66      0.38      2405
weighted avg       0.97      0.53      0.67      2405

FDR: 0.021250129834831766
[[1233 1121]
 [  10   41]]
channel:  S
              precision    recall  f1-score   support

           0       0.98      0.62      0.76       342
           1       0.13      0.80      0.23        25

    accuracy                           0.63 